In [194]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [33]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

In [34]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [35]:
!pip install -q findspark

In [37]:
!echo $SPARK_HOME

/content/spark-2.4.6-bin-hadoop2.7


In [39]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [40]:
import findspark
import pandas as pd
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [186]:
import pyspark.sql.functions as f
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DateConverter

In [195]:
# CSV file 
df=spark.read.csv("startup.csv" , header=True , inferSchema=True)

In [64]:
# paraquet file
df2=spark.read.parquet("/content/consumerInternet.parquet")

In [70]:
#Merging of both above file 
df3 = df.union(df2)

In [83]:
df3.show()

+-----+----------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+-------------+-------+
|Sr_No|      Date|        Startup_Name|   Industry_Vertical|         SubVertical|         City|      Investors_Name|     InvestmentnType|Amount_in_USD|Remarks|
+-----+----------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+-------------+-------+
|    1|09/01/2020|              BYJU’S|              E-Tech|          E-learning|    Bengaluru|Tiger Global Mana...|Private Equity Round| 20,00,00,000|   null|
|    2|13/01/2020|              Shuttl|      Transportation|App based shuttle...|      Gurgaon|Susquehanna Growt...|            Series C|    80,48,394|   null|
|    3|09/01/2020|           Mamaearth|          E-commerce|Retailer of baby ...|    Bengaluru|Sequoia Capital I...|            Series B|  1,83,58,860|   null|
|    4|02/01/2020|https://www.wealt...| 

In [139]:
df3.printSchema()

root
 |-- Sr_No: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Startup_Name: string (nullable = true)
 |-- Industry_Vertical: string (nullable = true)
 |-- SubVertical: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Investors_Name: string (nullable = true)
 |-- InvestmentnType: string (nullable = true)
 |-- Amount_in_USD: string (nullable = true)
 |-- Remarks: string (nullable = true)



Q1.How many startups are there in Pune City?

In [90]:

df3.select("city").where(col("city")=="Pune").count()

105

Q2.How many startups in Pune got their Seed/ Angel Funding?

In [101]:
#found Seed/Angle in csv file Checked manually 
df3.where("city=='Pune' and InvestmentnType like 'Seed%Ang%Funding'").count()


6

Q3.What is the total amount raised by startups in Pune City? 

In [253]:
#Amount in string so replaced ',' with ''
#Amount column string converted into integer
#changing nan values with mean
finaldf = df3.withColumn("No_Null_Amount_USD",f.regexp_replace("Amount_in_USD",',',''))
finaldf=finaldf.withColumn("No_Null_Amount_USD",finaldf["No_Null_Amount_USD"].cast(IntegerType()))
mean = finaldf.groupby().mean().collect()[0][0]
finaldf = finaldf.fillna(mean)

finaldf.agg(f.sum("No_Null_Amount_USD")).show()

+-----------------------+
|sum(No_Null_Amount_USD)|
+-----------------------+
|            46705526385|
+-----------------------+



Q4.What are the top 5 Industry_Vertical which has the highest number of startups in India?

In [181]:
#Found null value in Industry_Vertical column
df3.where("Industry_Vertical not like 'nan'").groupby("Industry_Vertical").count().sort(col('count').desc()).show(5)

+-----------------+-----+
|Industry_Vertical|count|
+-----------------+-----+
|Consumer Internet|  941|
|       Technology|  478|
|        eCommerce|  186|
|       Healthcare|   70|
|          Finance|   62|
+-----------------+-----+
only showing top 5 rows



Q5. Find the top Investor(by amount) of each year.

In [270]:
#'01/07/015' date 
#tried to change date string to timestamp and then retrieving only date column with year
tempdf = finaldf.select('Date', 'Investors_Name','No_Null_Amount_USD', f.from_unixtime(f.unix_timestamp('Date', 'dd/MM/yyyy')).alias('date_'))
tempdf.select(f.year("date_"),'*').groupby("date_").max("No_Null_Amount_USD").show()

Py4JJavaError: ignored

Bonus:
Q1 Find the top startup(by amount raised) from each city?

In [259]:
finaldf.groupby("City","Startup_Name").max("No_Null_Amount_USD").show()

+---------+--------------------+-----------------------+
|     City|        Startup_Name|max(No_Null_Amount_USD)|
+---------+--------------------+-----------------------+
|New Delhi|Bombay Shaving Co...|                2300000|
|     Pune|            Arcatron|               15343471|
|Bangalore|                Zapr|               10000000|
|   Mumbai|               Pokkt|                5000000|
|   Mumbai|                Fixy|               15343471|
|  Gurgaon|          Paper Boat|               30000000|
|      nan|           OYO Rooms|               25000000|
|   Mumbai|                Acko|               12000000|
|New Delhi|       91SpringBoard|               15343471|
|New Delhi|           HealthMir|               15343471|
|  Gurgaon|              Edurev|                  40000|
|Bangalore|           Justbooks|               15343471|
|   Mumbai|           AppsDaily|                3750000|
|Bangalore|               Indee|                 500000|
|   Mumbai|            Murgency